# An example to demonstrate online dynamic mode decomposition

**Original authors:**

- Hao Zhang
- Clarence W. Rowley

**Adapted by author:**

- Daniel Lehmberg

**Reference:**

Zhang, Hao, Clarence W. Rowley, Eric A. Deem, and Louis N. Cattafesta. "Online dynamic mode decomposition for time-varying systems." SIAM Journal on Applied Dynamical Systems 18, no. 3 (2019): 1586-1609. [doi:10.1137/18M1192329](https://doi.org/10.1137/18M1192329) ([preprint](https://arxiv.org/abs/1707.02876))

**See also:**

* the original  [odmd repository](https://github.com/haozhg/odmd) on github
* the original license (MIT) in datafold's [LICENSE_bundled](https://gitlab.com/datafold-dev/datafold/-/blob/master/LICENSES_bundled) file.




*Note that currently datafold's `OnlineDMD` adaptation performs a lot of checks on input data. This means that many single-sample updates on `partial_fit` in a loop are expensive. Adaptions for fast single-sample updates are considered for future development.*

-----

We take a 2D time varying system given by $dx/dt = A(t)x$ where $x = [x_1,x_2]^T, A(t) = [0, w(t); -w(t),0]$,
$w(t)=1+\varepsilon t$, $\varepsilon=0.1$. The slowly time varying eigenvlaues of $A(t)$ are pure imaginary, $+(1+0.1t)j$ and $-(1+0.1t)j$, where $j$ is the imaginary unit.

At time step $k$, define two matrix $X(k) = [x(1),x(2),...,x(k)]$, $Y(k) = [y(1),y(2),...,y(k)]$, that contain all the past snapshot pairs, we would like to compute $A_k = Y_k \cdot X_k^\dagger$. This can be done by brute-force
batch DMD, and by efficient rank-1 updating online DMD algrithm. Batch DMD computes DMD matrix by brute-force taking the pseudo-inverse directly. Online DMD computes the DMD matrix by using efficient rank-1 update idea.

We compare the performance of online DMD (with weighting=[1, 0.9]) with the brute-force batch DMD approach in terms of tracking time varying eigenvalues, by comparison with the analytical solution. Online DMD (weighting=1) and
batch DMD should agree with each other (up to machine round-off errors).)


In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint

from datafold import TSCDataFrame
from datafold.dynfold.dmd import OnlineDMD

In [ ]:
# define dynamics
epsilon = 1e-1


def dyn(x, t):
    x1, x2 = x
    dxdt = [(1 + epsilon * t) * x2, -(1 + epsilon * t) * x1]
    return dxdt


# integrate from initial condition [1,0]
tspan = np.linspace(0, 10, 101)
dt = 0.1
x0 = [1, 0]
xsol = odeint(dyn, x0, tspan).T
# extract snapshots
x, y = xsol[:, :-1], xsol[:, 1:]
X = TSCDataFrame.from_shift_matrices(left_matrix=x, right_matrix=y)

t = tspan[1:]
# true dynamics, true eigenvalues
n, m = len(x[:, 0]), len(x[0, :])
A = np.empty((n, n, m))
evals = np.empty((n, m), dtype=complex)
for k in range(m):
    A[:, :, k] = np.array([[0, (1 + epsilon * t[k])], [-(1 + epsilon * t[k]), 0]])
    evals[:, k] = np.linalg.eigvals(A[:, :, k])

# visualize snapshots
plt.figure(figsize=(10, 6.6))
plt.plot(tspan, xsol[0, :], "b-", linewidth=2.0, label="x_1(t)")
plt.plot(tspan, xsol[1, :], "g--", linewidth=2.0, label="x_2(t)")
plt.legend(loc="best", fontsize=12)
plt.xlabel("Time", fontsize=12)
plt.title("State", fontsize=12)
plt.tick_params(labelsize=12)
plt.grid()

In [ ]:
# batch DMD
q = 10
AbatchDMD = np.empty((n, n, m))
evalsbatchDMD = np.empty((n, m), dtype=complex)
start = time.time()
for k in range(q, m):
    AbatchDMD[:, :, k] = y[:, : k + 1].dot(np.linalg.pinv(x[:, : k + 1]))
    evalsbatchDMD[:, k] = np.log(np.linalg.eigvals(AbatchDMD[:, :, k])) / dt
end = time.time()
print("Batch DMD, time = " + str(end - start) + " secs")

init_X = X.loc[pd.IndexSlice[:q, :], :]

# Online DMD, weighting = 1
evalsonlineDMD1 = np.empty((n, m), dtype=complex)
odmd = OnlineDMD(weighting=1.0)
odmd = odmd.partial_fit(init_X, batch_fit=True)

start = time.time()
for k in range(q, m):
    odmd.partial_fit(X.loc[[k], :])
    evalsonlineDMD1[:, k] = np.log(np.linalg.eigvals(odmd.A)) / dt
end = time.time()
print(f"{odmd=}, time = " + str(end - start) + " secs")


# Online DMD, weighting = 0.9
evalsonlineDMD2 = np.empty((n, m), dtype=complex)
odmd = OnlineDMD(weighting=0.9)
odmd = odmd.partial_fit(init_X, batch_fit=True)
start = time.time()
for k in range(q, m):
    odmd.partial_fit(X.loc[[k], :])
    evalsonlineDMD2[:, k] = np.log(np.linalg.eigvals(odmd.A)) / dt
end = time.time()
print(f"{odmd=}, time = " + str(end - start) + " secs")

# Online DMD, weighting = 0.4
evalsonlineDMD3 = np.empty((n, m), dtype=complex)
odmd = OnlineDMD(weighting=0.4)
odmd = odmd.partial_fit(init_X, batch_fit=True)
start = time.time()
for k in range(q, m):
    odmd.partial_fit(X.loc[[k], :])
    evalsonlineDMD3[:, k] = np.log(np.linalg.eigvals(odmd.A)) / dt
end = time.time()
print(f"{odmd=}, time = " + str(end - start) + " secs")

# visualize true, batch, online (weighting=[1,0.9, 0.4])
plt.figure(figsize=(10, 6.6))
plt.title("Frequency", fontsize=12)
plt.plot(t, np.imag(evals[0, :]), "k-", label="True", linewidth=2.0)
plt.plot(
    t[q:],
    np.imag(evalsbatchDMD[0, q:]),
    "r--",
    label="Batch (brute force), wf=1",
    linewidth=2.0,
)
plt.plot(
    t[q:],
    np.imag(evalsonlineDMD1[0, q:]),
    "g-.",
    alpha=0.5,
    label="Online, wf=1",
    linewidth=2.0,
)
plt.plot(
    t[q:], np.imag(evalsonlineDMD2[0, q:]), "b:", label="Online, wf=0.9", linewidth=2.0
)
plt.plot(
    t[q:], np.imag(evalsonlineDMD3[0, q:]), "r-", label="Online, wf=0.4", linewidth=2.0
)
plt.tick_params(labelsize=12)
plt.xlabel("time", fontsize=12)
plt.ylabel(r"Im(lambda_DMD)", fontsize=12)
plt.legend(loc="best", fontsize=12)
plt.show()